# Selection

> This module contains all of the necessary code for creating the first stage of the pipeline needed for a sequential UI. For more on the technical aspects, see the [Panel documentation](https://panel.holoviz.org/how_to/pipeline/simple_pipeline.html) on ```Pipeline```. This tab includes an info pane, a selection widget, and two buttons (select and new manuscript buttons).

In [1]:
#| default_exp selection

In [2]:
#| hide
from nbdev.showdoc import *
from fastcore.utils import *

This module relies on the widgetCalls and mansucriptFiles modules to work. This is the import statement for those libraries.

In [3]:
#| export
import ipywidgets as widgets
import param
import panel as pn
pn.extension('ipywidgets')

from glyptodon.widgetCalls import *
from glyptodon.manuscriptFiles import currentManuscripts

First, we want to define a function call that creates a selection widget so that we can put it into the Selection Stage later. This widget needs the current manuscripts to be available when defining the selection options. For that, we call ```currentManuscripts```. The output of ```currentManuscripts``` includes the directory information for different manuscripts, so we want that output available within our Selection Stage. That is why this returns ```manuscripts```, ```selectionKey```, and the ```manuscriptSelect``` widget itself.

In [4]:
#| export
def createManuscriptSelect():
    manuscripts = currentManuscripts()
    
    selectionKey = {}
    selectionNames = []
    for manuscript in manuscripts:
        selectionNames.append(manuscript[1]['Work'])
        selectionKey[selectionNames[-1]] = manuscript
        
    manuscriptSelect = widgets.Select(options = selectionNames,
                                      value = selectionNames[0],
                                      rows = 10,
                                      disabled = False,
                                      layout = widgets.Layout(height = 'auto',
                                                              width = '425px'
                                                             )
                                     )
    
    return selectionKey, manuscriptSelect

In [6]:
#| hide

# This is testing that the correct widgets and manuscripts are displaying, which they are.
[keys, widge] = createManuscriptSelect()

widge

Select(layout=Layout(height='auto', width='425px'), options=('Data', 'Nicomachean Ethics', 'Republic'), rows=1…

In [7]:
widge.value

'Data'

Now we need to have an info pane that tells users what to do when using the Selection Stage. This function creates an info pane and returns it.

In [38]:
#| export
def createSelectionInfo():
    return widgets.HTML(value = "<h2>Info Pane</h2>\
                                <p>This menu allows you to upload new manuscripts and access previously uploaded manuscripts.\
                                The uploader will accept only .xml and image files.\
                                For testing purposes, it only accepts images right now.<p>",
                       layout = widgets.Layout(max_width = '600px')
                       )

In [39]:
#| hide

createSelectionInfo()

HTML(value='<h2>Info Pane</h2>                                <p>This menu allows you to upload new manuscript…

Now we want to fit all of this into one pane that can be used as Stage. This is the place where all of the widgets are put together into a cohesive whole. This first example only shows the image. We will still need to have observers for changes in the widgets.

In [40]:
#| hide
class LayoutDemo(param.Parameterized):
    
    def panel(self):
        self.newMan = tealButton('New Manuscript')
        self.selMan = blueButton('Select Manuscript')

        buttons = pn.Row(self.selMan, self.newMan)

        self.selectionKey, self.manuscriptSelect = createManuscriptSelect()

        selections = pn.Column(self.manuscriptSelect, buttons)
        return pn.Row(createSelectionInfo(), selections)

In [41]:
#| hide

layoutDemo = LayoutDemo()

layoutDemo.panel()

Row
    [0] IPyWidget(HTML)
    [1] Column
        [0] IPyWidget(Select)
        [1] Row
            [0] IPyWidget(Button)
            [1] IPyWidget(Button)

Note that the traits of the objects are available to call. Most importantly, we can access the value of the ```manuscriptSelect``` widget.

In [12]:
#| hide

print(layoutDemo.selectionKey[layoutDemo.manuscriptSelect.value])

['/home/dc/glyptodon/manuscripts/data', {'Work': 'Data', 'Author': 'Euclid', 'Language': 'Greek', 'Country': 'Egpyt', 'City': 'Alexandria', 'Institution': 'The Great Library', 'Centuries': '3rd century BC'}]


Now that the layout is done, we need to have methods linked to the buttons that will change what data is output from this stage of the ```Pipeline``` and pushes it into the next stage, Manuscript Info. To do this, we need to flesh out some of the ```param``` functionality of the ```Pipeline```. We need output variables built in. So this is a rework of ```SelectionDemo``` into ```Selection```.

In [67]:
#| export
class Selection(param.Parameterized):
    
    newManClicked = False
    # Set this to a default manuscript ASAP
    selectedManuscript = list()
    
    # This is the output for the class
    @param.output(('newManClicked', param.Boolean),('selectedManuscript', param.List))
    def selectionOutput(self):
        return self.newManClicked, self.selectedManuscript
    
    
    # This functions like the standard __init__ function
    def panel(self):
        self.newMan = tealButton('Create Manuscript')
        self.selMan = blueButton('Select Manuscript')
        
        # Binding buttons to functions that will be patched later
        self.newMan.on_click(self.on_click_newMan)
        self.selMan.on_click(self.on_click_selMan)
        
        buttons = pn.Row(self.selMan, self.newMan)

        self.selectionKey, self.manuscriptSelect = createManuscriptSelect()

        selections = pn.Column(self.manuscriptSelect, buttons)
        return pn.Row(createSelectionInfo(), selections)

We will start with a method that handles on click events for the ```selMan``` button. Note that this patches to ```Selection```. That class will make this function fully available to the same layout as ```SelectionDemo```. This function, ```on_click_selMan```, sets the output value of ```selectedManuscript``` to match the value of the ```manuscriptSelect``` widget

In [68]:
#| export
@patch
def on_click_selMan(self:Selection, null):
    if len(self.selectedManuscript) == 0:
        self.selectedManuscript = self.selectionKey[self.manuscriptSelect.value]
        self.selMan.description = 'Select Different Manuscript'
    else:
        self.selectedManuscript = self.selectionKey[self.manuscriptSelect.value]

Next, we want to handle click events for ```newMan```. This to will be a patched function. It will set the value ```newManClicked``` to to ```True``` so that the next stage can be modified accordingly.

In [69]:
#| export
@patch
def on_click_newMan(self:Selection, null):
    if self.newManClicked == False:
        self.newManClicked = True
        self.newMan.description = "Don't Create Manuscript"
    else:
        self.newManClicked = False
        self.newMan.description = 'Create Manuscript'

Now you can test out whether or not this works. Below is an instance of the panel and some print statements.

In [70]:
#| hide

selectionDemo = Selection()

selectionDemo.panel()

Row
    [0] IPyWidget(HTML)
    [1] Column
        [0] IPyWidget(Select)
        [1] Row
            [0] IPyWidget(Button)
            [1] IPyWidget(Button)

In [71]:
#| hide
print(selectionDemo.newManClicked)
print(selectionDemo.selectedManuscript)

False
['/home/dc/glyptodon/manuscripts/republic', {'Work': 'Republic', 'Author': 'Plato', 'Language': 'Greek', 'Country': 'Greece', 'City': 'Athens', 'Institution': 'The Academy', 'Centuries': '4th century BC'}]


In [64]:
#| hide
selectionDemo.param.outputs()

{'newManClicked': (<param.Boolean at 0x7fb44ff55bc0>,
  <bound method Selection.selectionOutput of Selection(name='Selection00466')>,
  0),
 'selectedManuscript': (<param.List at 0x7fb44ff51f30>,
  <bound method Selection.selectionOutput of Selection(name='Selection00466')>,
  1)}

In [66]:
#| hide
selectionDemo.selectionOutput()

(True,
 ['/home/dc/glyptodon/manuscripts/nicomacheanethics',
  {'Work': 'Nicomachean Ethics',
   'Author': 'Aristotle',
   'Language': 'Greek',
   'Country': 'Greece',
   'City': 'Athens',
   'Institution': 'The Academy',
   'Centuries': '4th century BC'}])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()